In [45]:
import onnx
import tensorflow
import sys
from os.path import dirname
sys.path.append(dirname("onnx-tensorflow/"))
from onnx_tf.backend import prepare

onnx_model = onnx.load("chest.onnx")  # load onnx model
onnx.checker.check_model(onnx_model)

In [46]:
from onnx import optimizer
# all_passes = optimizer.get_available_passes()
# print("Available optimization passes:")
# for p in all_passes:
#     print(p)
# print()

In [47]:
# passes = ["eliminate_identity", 
#           "eliminate_unused_initializer", 
#           "extract_constant_to_initializer",
#           "split_init",
#           "nop",
#           "fuse_consecutive_squeezes",
#           "fuse_consecutive_transposes",
#           "eliminate_nop_pad"]
#optimized_model = optimizer.optimize(onnx_model, passes)

#print('The model after optimization:\n{}'.format(optimized_model))

# One can also apply the default passes on the (serialized) model
# Check the default passes here: https://github.com/onnx/onnx/blob/master/onnx/optimizer.py#L43
optimized_model = optimizer.optimize(onnx_model)

In [48]:
#print('The model after optimization:\n{}'.format(optimized_model))

In [49]:
tf_rep = prepare(optimized_model)  # prepare tf representation
tf_rep.export_graph("chest.pb")  # export the model

In [37]:
import tensorflow as tf
from __future__ import print_function
f = tf.gfile.GFile("chest.pb", 'rb')
graph_def = tf.GraphDef()
graph_def.ParseFromString(f.read())
a = tf.import_graph_def(graph_def)

In [38]:
def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we can use again a convenient built-in function to import a graph_def into the 
    # current default Graph
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            graph_def, 
            input_map=None, 
            return_elements=None, 
            name="prefix", 
            op_dict=None, 
            producer_op_list=None
        )
    return graph

In [39]:
graph = load_graph("chest.pb")

In [40]:
for op in graph.get_operations():
    print(op.name)

prefix/Const
prefix/Const_1
prefix/Const_2
prefix/Const_3
prefix/Const_4
prefix/Const_5
prefix/Const_6
prefix/Const_7
prefix/Const_8
prefix/Const_9
prefix/Const_10
prefix/Const_11
prefix/Const_12
prefix/Const_13
prefix/Const_14
prefix/Const_15
prefix/actual_input_1
prefix/transpose/perm
prefix/transpose
prefix/Const_16
prefix/Pad
prefix/Const_17
prefix/split/split_dim
prefix/split
prefix/transpose_1/perm
prefix/transpose_1
prefix/Const_18
prefix/split_1/split_dim
prefix/split_1
prefix/convolution/dilation_rate
prefix/convolution
prefix/concat/concat_dim
prefix/concat
prefix/Add
prefix/transpose_2/perm
prefix/transpose_2
prefix/Relu
prefix/transpose_3/perm
prefix/transpose_3
prefix/max_pool/dilation_rate
prefix/max_pool
prefix/transpose_4/perm
prefix/transpose_4
prefix/transpose_5/perm
prefix/transpose_5
prefix/Const_19
prefix/Pad_1
prefix/Const_20
prefix/split_2/split_dim
prefix/split_2
prefix/transpose_6/perm
prefix/transpose_6
prefix/Const_21
prefix/split_3/split_dim
prefix/split_3
p

In [42]:
#!rm -rf test2

In [43]:
# !git clone https://github.com/tensorflow/tfjs-converter
!tensorflowjs_converter \
    --input_format=tf_frozen_model \
    --output_node_names='add_8' \
    chest.pb \
    test2


#!tensorflowjs_converter --input_format= --output_node_names=

Using TensorFlow backend.
2018-12-24 20:40:37.666684: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:501] Optimization results for grappler item: graph_to_optimize
2018-12-24 20:40:37.666749: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:503]   debug_stripper: Graph size after: 228 nodes (0), 212 edges (0), time = 195.869ms.
2018-12-24 20:40:37.666756: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:503]   model_pruner: Graph size after: 200 nodes (-28), 207 edges (-5), time = 416.625ms.
2018-12-24 20:40:37.666762: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:503]   constant folding: Graph size after: 121 nodes (-79), 123 edges (-84), time = 7969.9751ms.
2018-12-24 20:40:37.666768: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:503]   arithmetic_optimizer: Graph size after: 98 nodes (-23), 129 edges (6), time = 802.748ms.
2018-12-24 20:40:37.666774: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:503]   dependency_optimizer: Graph 

In [44]:
!scp -r test2 jcccnetc@jcccnet.com:www/mlmed/tools/xray3/

group1-shard4of59                             100% 4096KB 215.1KB/s   00:19    
group1-shard59of59                            100% 1107KB 221.0KB/s   00:05    
group1-shard58of59                            100% 4096KB 227.0KB/s   00:18    
group1-shard5of59                             100% 4096KB 227.0KB/s   00:18    
group1-shard7of59                             100% 4096KB 226.9KB/s   00:18    
group1-shard6of59                             100% 4096KB 227.0KB/s   00:18    
group1-shard2of59                             100% 4096KB 226.9KB/s   00:18    
group1-shard3of59                             100% 4096KB 227.1KB/s   00:18    
group1-shard1of59                             100% 4096KB 227.0KB/s   00:18    
group1-shard48of59                            100% 4096KB 227.1KB/s   00:18    
group1-shard49of59                            100% 4096KB 227.0KB/s   00:18    
group1-shard12of59                            100% 4096KB 215.0KB/s   00:19    
weights_manifest.json                   